In [11]:
import pandas as pd

train_data = pd.read_csv('data\labeled_final_train.csv')
test_data = pd.read_csv('data\labeled_final_test.csv')

In [6]:
train_data.head()
# len(train_data)

,id,sentence1,sentence2,label
0,1,"In Paris , in October 1560 , he secretly met t...","In October 1560 , he secretly met with the Eng...",0
1,2,The NBA season of 1975 -- 76 was the 30th seas...,The 1975 -- 76 season of the National Basketba...,1
2,3,"There are also specific discussions , public p...","There are also public discussions , profile sp...",0
3,4,When comparable rates of flow can be maintaine...,The results are high when comparable flow rate...,1
4,5,It is the seat of Zerendi District in Akmola R...,It is the seat of the district of Zerendi in A...,1


In [13]:
from torch.utils.data import Dataset
import torch

In [9]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [18]:
import torch
import pandas as pd
from transformers import BertTokenizer
from torch.utils.data import Dataset, DataLoader

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

# Load CSV file
data = pd.read_csv('data/labeled_final_test.csv')

# Custom Dataset class
class ParaphraseDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]

        # Tokenize both sentences
        encoding1 = self.tokenizer(
            row["sentence1"], padding="max_length", truncation=True, 
            max_length=self.max_length, return_tensors="pt"
        )
        encoding2 = self.tokenizer(
            row["sentence2"], padding="max_length", truncation=True, 
            max_length=self.max_length, return_tensors="pt"
        )

        # Extract input tensors
        input_ids1 = encoding1["input_ids"].squeeze(0)  # Shape: (seq_len,)
        input_ids2 = encoding2["input_ids"].squeeze(0)  # Shape: (seq_len,)

        # Convert label to tensor
        label = torch.tensor(row["label"], dtype=torch.long)

        return (input_ids1, input_ids2, label)

# Create Dataset
dataset = ParaphraseDataset(data, tokenizer)

# Create DataLoader
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# Example batch
for batch in dataloader:
    sentence1, sentence2, labels = batch
    print("Sentence 1 batch shape:", sentence1.shape)  # (batch_size, seq_len)
    print("Sentence 2 batch shape:", sentence2.shape)  # (batch_size, seq_len)
    print("Labels shape:", labels.shape)  # (batch_size,)
    print(sentence1)
    break


Sentence 1 batch shape: torch.Size([8, 128])
Sentence 2 batch shape: torch.Size([8, 128])
Labels shape: torch.Size([8])
tensor([[  101,  1130,  1754,  ...,     0,     0,     0],
        [  101,  4101,  2921,  ...,     0,     0,     0],
        [  101,  1109, 11730,  ...,     0,     0,     0],
        ...,
        [  101, 12556,  6582,  ...,     0,     0,     0],
        [  101,   148,  7921,  ...,     0,     0,     0],
        [  101,  9800,  1233,  ...,     0,     0,     0]])


In [5]:
import torch
logits1 = torch.randn(size=(12,128))
logits2 = torch.randn(size=(12,128))
similarity = torch.tensordot(logits1,logits2,dims=([1,1],))

ValueError: not enough values to unpack (expected 2, got 1)